In [4]:
# Preflight Playwright "solo uv" (no usa pip)
import os, sys, subprocess, pathlib, platform

print("Kernel Python:", sys.executable)

# (Opcional) fuerza una ruta persistente para los browsers
browsers_path = pathlib.Path.home() / ".cache" / "ms-playwright"
browsers_path.mkdir(parents=True, exist_ok=True)
os.environ["PLAYWRIGHT_BROWSERS_PATH"] = str(browsers_path)

# Descarga Chromium usando el MISMO intérprete del kernel
rc = subprocess.run([sys.executable, "-m", "playwright", "install", "chromium"]).returncode
if rc != 0 and platform.system() == "Linux":
    print("Si estás en una distro Arch/Cachy y faltan libs del SO, instala dependencias del sistema con pacman, o usa contenedor.")

# Verificación
found = list(browsers_path.glob("chromium-*/*/chrome*"))
print("Chromium instalado en:", found[0] if found else "NO ENCONTRADO")
assert found, "No se encontró Chromium. Revisa permisos o instala vía terminal con uvx."


Kernel Python: /home/lynn/Documentos/development/scripts-notebooks/professional-projects/ajover/.venv/bin/python
BEWARE: your OS is not officially supported by Playwright; downloading fallback build for ubuntu20.04-x64.
|                                                                                |   0% of 171.6 MiB
|■■■■■■■■                                                                        |  10% of 171.6 MiB
|■■■■■■■■■■■■■■■■                                                                |  20% of 171.6 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■                                                        |  30% of 171.6 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                                                |  40% of 171.6 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                                        |  50% of 171.6 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                                |  60% of 171.6 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                

In [7]:
from scrapegraphai.graphs import SmartScraperMultiGraph
from pydantic import BaseModel, Field
import os
import nest_asyncio
import copy
import hashlib
import base64
import unicodedata

nest_asyncio.apply()

# ─────────────────────────────────────────────────────────────
# 1) Esquema pydantic para validar la extracción
# ─────────────────────────────────────────────────────────────
description = (
    "Las claves deben ser los títulos de las secciones encontrados en la página. "
    "Los valores deben ser el texto completo correspondiente a cada sección, sin resumir."
)

class ContenidoDinamico(BaseModel):
    contenido: dict[str, str] = Field(description=description)

# ─────────────────────────────────────────────────────────────
# 2) Mapeo de nombre a URL (nombres amigables)
# ─────────────────────────────────────────────────────────────
sources = {
    "sobre ajover": "https://www.ajover.com/sobre-ajover/",
    "sostenibilidad": "https://www.ajover.com/sostenibilidad/",
    "noticias": "https://www.ajover.com/noticias/",
    "proyectos": "https://www.ajover.com/proyectos/",
    "gana ajover": "https://www.ajover.com/gana-ajover/",
    "match": "https://www.ajover.com/noticia/crea-match-perfectos-con-la-mejor-calidad-ajover/",
    "aumentar ventas": "https://www.ajover.com/noticia/aumenta-tus-ventas-con-nuestra-cubierta-termoacustica/",
    "renovar espacios": "https://www.ajover.com/noticia/renueva-los-espacios-y-que-tu-hogar-sea-tu-lugar-favorito/",
    "construccion sostenible": "https://www.ajover.com/noticia/ajover-se-une-al-consejo-colombiano-de-construccion-sostenible/",
}

# ─────────────────────────────────────────────────────────────
# 3) Config de ScrapegraphAI
# ─────────────────────────────────────────────────────────────
config = {
    "llm": {
        "api_key": os.getenv("AZURE_OPENAI_API_KEY"),
        "model": "azure_openai/gpt-4o",
        "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
    },
    "verbose": False,
    "headless": True,
}

# ─────────────────────────────────────────────────────────────
# 4) Utilidades
# ─────────────────────────────────────────────────────────────
def _norm(s: str) -> str:
    """Normaliza nombre de fuente: minúsculas, sin tildes, espacios/guiones→_."""
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode()
    s = s.lower().strip().replace(" ", "_").replace("-", "_")
    while "__" in s:
        s = s.replace("__", "_")
    return s

def generate_deterministic_id(input_str: str, max_length: int = 40) -> str:
    hash_bytes = hashlib.sha256(input_str.encode("utf-8")).digest()
    base64_bytes = base64.urlsafe_b64encode(hash_bytes)
    clean_id = base64_bytes.decode("utf-8").rstrip("=")
    return clean_id[:max_length]

def convertir_a_documentos_web(dict_preprocesado: dict) -> list[dict]:
    """
    Convierte el diccionario procesado por página a una lista de documentos listos para indexar.
    - Omite claves 'url'/'error'
    - Omite valores vacíos o 'NA'
    """
    documentos: list[dict] = []
    for nombre_fuente, secciones in dict_preprocesado.items():
        if not isinstance(secciones, dict):
            continue
        if "error" in secciones:
            continue
        url = secciones.get("url", None)
        for titulo, texto in secciones.items():
            if titulo in {"url", "error"}:
                continue
            if texto is None:
                continue
            if isinstance(texto, str) and texto.strip().upper() == "NA":
                continue
            doc_id = generate_deterministic_id(f"{nombre_fuente}_{titulo}")
            documentos.append(
                {
                    "id": doc_id,
                    "titulo": str(titulo),
                    "contenido": str(texto),
                    "fuente": nombre_fuente,
                    "url": url,
                }
            )
    return documentos

# ─────────────────────────────────────────────────────────────
# 5) Ejecución del scraping
# ─────────────────────────────────────────────────────────────
results: dict[str, dict] = {}

for name, url in sources.items():
    prompt = (
        f"Analiza el contenido de la página {url} y extrae toda su información textual de manera estructurada. "
        "Identifica cada sección principal por sus encabezados (h1, h2, h3, etc.) "
        "y para cada sección extrae TODO el texto literal y completo, sin resumir. "
        "Devuelve un objeto JSON conforme al esquema."
    )
    scraper = SmartScraperMultiGraph(
        prompt=prompt,
        source=[url],
        config=config,
        schema=ContenidoDinamico,
    )
    try:
        result = scraper.run()
        contenido = None
        # result puede ser un Pydantic model o un dict
        if hasattr(result, "contenido"):
            contenido = dict(result.contenido)
        elif isinstance(result, dict):
            contenido = result.get("contenido", result)
        if isinstance(contenido, dict):
            contenido["url"] = url
            results[name] = contenido
        else:
            results[name] = {"error": f"Formato inesperado: {type(result)}", "url": url}
    except Exception as e:
        results[name] = {"error": str(e), "url": url}

# ─────────────────────────────────────────────────────────────
# 6) Normalización de nombres de fuente y preprocesamiento
# ─────────────────────────────────────────────────────────────
# Normaliza claves de results (para evitar 'sobre ajover' vs 'sobre_ajover')
results_norm = {_norm(k): v for k, v in results.items()}
dict_preprocesado = copy.deepcopy(results_norm)

# Garantiza que exista el destino 'sobre_ajover' aunque la página falle
dict_preprocesado.setdefault("sobre_ajover", {})

# Secciones a mantener y a eliminar (por título de sección)
mantener = {"Contacto", "Copyright"}
eliminar = {
    "Línea Ética",
    "Politica de tratamiento de datos",
    "Bodega Inducol",
    "Urbanización San Carlos II",
    "Suscríbete a nuestro newsletter",
    "Links relevantes",
    "Casa Campestre Delicias",
    "Casa Campestre Villa Leo",
    "Casa residencial",
}

# Prioridad de fuentes (normalizadas)
prioridad = ["sobre_ajover", "sostenibilidad", "noticias", "proyectos", "gana_ajover"]

# Paso 0: elimina entradas con error para no propagar basura
dict_preprocesado = {
    k: v for k, v in dict_preprocesado.items() if isinstance(v, dict) and "error" not in v
}

# Paso 1: Mover todas las claves de mantener a 'sobre_ajover'
for key in mantener:
    for name, secciones in list(dict_preprocesado.items()):
        if name == "sobre_ajover":
            continue
        if isinstance(secciones, dict) and key in secciones:
            dict_preprocesado["sobre_ajover"].setdefault(key, secciones[key])
            del secciones[key]

# Paso 2: Eliminar todas las claves de eliminar en todos los diccionarios
for seccion in dict_preprocesado.values():
    if isinstance(seccion, dict):
        for key in eliminar:
            seccion.pop(key, None)

# Paso 3: Elimina claves duplicadas según prioridad (excepto "url")
vistos: set[str] = set()
for name in prioridad:
    d = dict_preprocesado.get(name, {})
    if not isinstance(d, dict):
        continue
    for key in list(d.keys()):
        if key == "url":
            continue
        if key in vistos:
            del d[key]
        else:
            vistos.add(key)

# ─────────────────────────────────────────────────────────────
# 7) Conversión a documentos de indexación
# ─────────────────────────────────────────────────────────────
documentos_web = convertir_a_documentos_web(dict_preprocesado)

{'contenido': {'¿Quiénes somos?': 'Somos una compañía colombiana con más de 60 años de presencia en el mercado de la construcción, siendo líderes en innovación y tecnología plasmada en cada uno de nuestros productos, esto nos ha consolidado como una de las marcas más queridas a nivel nacional. Actualmente, contamos con plantas en Cartagena y Madrid, donde fabricamos y distribuimos nuestro gran portafolio de productos a nivel nacional, el cual incluye cubiertas termoacústicas, tejas de policarbonato, láminas de policarbonato alveolar, láminas para divisiones de espacios, tanques multiusos, barreras y estibas, láminas arquitectónicas y sistemas para tratamiento de aguas residuales. En Ajover Darnel S.A.S. trabajamos incansablemente para que cada uno de nuestros productos para la construcción estén siempre enfocados en mejorar la calidad de vida de todos los clientes.', 'Misión': 'Ofrecer a nuestros clientes productos y conveniencia excepcionales al mejor valor posible, buscando sostenibi

In [15]:
from collections import Counter

ids = [d["id"] for d in documentos_web]
assert len(ids) == len(set(ids)), "Hay IDs duplicados"
print("Total a indexar:", len(documentos_web))
print("Por fuente:", Counter(d["fuente"] for d in documentos_web))

Total a indexar: 42
Por fuente: Counter({'sostenibilidad': 8, 'aumentar_ventas': 7, 'match': 6, 'sobre_ajover': 5, 'proyectos': 4, 'renovar_espacios': 4, 'noticias': 3, 'construccion_sostenible': 3, 'gana_ajover': 2})


In [47]:
documentos_web

[{'id': 'AeMTka0Y3AlqPw4XtTNDaSu0WBHSZ7gklJgB7V8M',
  'titulo': '¿Quiénes somos?',
  'contenido': 'Somos una compañía colombiana con más de 60 años de presencia en el mercado de la construcción, siendo líderes en innovación y tecnología plasmada en cada uno de nuestros productos, esto nos ha consolidado como una de las marcas más queridas a nivel nacional. Actualmente, contamos con plantas en Cartagena y Madrid, donde fabricamos y distribuimos nuestro gran portafolio de productos a nivel nacional, el cual incluye cubiertas termoacústicas, tejas de policarbonato, láminas de policarbonato alveolar, láminas para divisiones de espacios, tanques multiusos, barreras y estibas, láminas arquitectónicas y sistemas para tratamiento de aguas residuales. En Ajover Darnel S.A.S. trabajamos incansablemente para que cada uno de nuestros productos para la construcción estén siempre enfocados en mejorar la calidad de vida de todos los clientes.',
  'fuente': 'sobre_ajover',
  'url': 'https://www.ajover